In [17]:
# importing dependencies
import pandas as pd
import glob
import os
import requests
from config import GEO_KEY

In [12]:
# setting path
files = os.path.join("Resources/RAW/Schools/greatschools*.csv")

In [13]:
#list of merged files returned
files = glob.glob(files)

In [14]:
# joining files 
school_df = pd.concat(map(pd.read_csv, files), ignore_index=True)
school_df.head(2)

,circle-rating--small,name,address,tablescraper-selected-row 2,tablescraper-selected-row 3,tablescraper-selected-row 4,tablescraper-selected-row href,rating-circle-small,scale,name href,tablescraper-selected-row,tablescraper-selected-row 6,tablescraper-selected-row href 2
0,10.0,Bird Rock Elementary School,"5371 La Jolla Hermosa Ave., La Jolla, CA, 92037",K-5,450,20:1,https://www.greatschools.org/california/la-jol...,NaN,NaN,NaN,NaN,NaN,NaN
1,9.0,Preuss School Ucsd,"9500 Gilman Drive, La Jolla, CA, 92093",6-12,836,20:1,https://www.greatschools.org/california/la-jol...,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
school_df.drop(['rating-circle-small','tablescraper-selected-row 2','tablescraper-selected-row 3','tablescraper-selected-row 4','tablescraper-selected-row href','tablescraper-selected-row 6','tablescraper-selected-row href 2','name href','tablescraper-selected-row'], axis=1, inplace=True)
school_df.dropna(axis=0, how='any', inplace=True)
school_df['Zip Code'] = school_df['address'].str[-5:]
school_df.set_index('Zip Code',inplace=True)
school_df

,circle-rating--small,name,address,scale
Zip Code,,,,
92011,9.0,Pacific Rim Elementary School,"1100 Camino De Las Ondas, Carlsbad, CA, 92011",Above average
92011,9.0,Aviara Oaks Middle School,"6880 Ambrosia Lane, Carlsbad, CA, 92011",Above average
92010,9.0,Sage Creek High School,"3900 Cannon Road, Carlsbad, CA, 92010",Above average
92011,9.0,Aviara Oaks Elementary School,"6900 Ambrosia Lane, Carlsbad, CA, 92011",Above average
92008,9.0,Kelly Elementary School,"4885 Kelly Drive, Carlsbad, CA, 92008",Above average
...,...,...,...,...
91901,7.0,The Heights Charter,"2710 Alpine Boulevard, Suite E, Alpine, CA, 91901",Above average
91901,6.0,Boulder Oaks Elementary School,"2320 Tavern Road, Alpine, CA, 91901",Average
91901,6.0,Joan Macqueen Middle School,"2001 Tavern Road, Alpine, CA, 91901",Average


In [18]:
# link example format
# https://maps.googleapis.com/maps/api/geocode/json?address=1600+Amphitheatre+Parkway,+Mountain+View,+CA&key=YOUR_API_KEY
school_coords = []
key = GEO_KEY
addresses = school_df['address'].tolist()
for addy in addresses: 
    
    # cleaning addresses for URL
    zips = addy[-7:]
    new_addy = addy.strip(zips)
    new_addy = new_addy.replace(" ", "+")
    new_addy = new_addy.replace(",", "")
    
    # URL for google geocode API
    school_link = f"https://maps.googleapis.com/maps/api/geocode/json?address={new_addy}&key={key}"
    
    # Make a 'Get' request for the school location data.
    school_location = requests.get(school_link)
    
    # Get the JSON data.
    school_data = school_location.json()
    lat = school_data["results"][0]["geometry"]["location"]["lat"]
    lng = school_data["results"][0]["geometry"]["location"]["lng"]
    
#   print(new_addy)
#   print(school_link)
#   print(school_location)
#   print(lat)
#   print(lng)
#   print()

    school_coords.append({"lat": lat,
                         "lng": lng})

In [26]:
new_school_df = pd.DataFrame(school_coords, index=None, columns = ["lat", "lng"])
new_school_df

,lat,lng
0,33.110238,-117.305166
1,33.103991,-117.277164
2,33.158706,-117.285339
3,33.104014,-117.277471
4,33.148428,-117.311449
...,...,...
588,32.835301,-116.755216
589,32.818612,-116.777682
590,32.819861,-116.778027
591,32.832502,-116.817095


In [27]:
school_df_merged = pd.merge(school_df, new_school_df, left_index=True, right_index=True)
school_df_merged

,circle-rating--small,name,address,scale,lat,lng


In [6]:
from pathlib import Path  
filepath = Path('Resources/Clean/San_Diego_School_Data.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
school_df.to_csv(filepath)